In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import certifi, os, ssl, requests

custom_ca_filepath = r"\\pmbelz.de\general\SYS\Certificates\AEB RootCA v2.pem"


os.environ['REQUESTS_CA_BUNDLE'] = custom_ca_filepath


if hasattr(ssl, 'create_default_context'):
    ssl_context = ssl.create_default_context(cafile=certifi.where())
    ssl_context.load_verify_locations(cafile=custom_ca_filepath)
    requests.packages.urllib3.util.ssl_.create_default_context = lambda: ssl_context

In [ ]:
import tkinter as tk
from tkinter import filedialog

def pick_file():
    root = tk.Tk()
    # root.withdraw()
    file_path = filedialog.askopenfilename()
    return file_path

# location of the pdf file/files. 
# reader = pick_file()

In [ ]:
new_file = pick_file()

In [ ]:
import PyPDF2

# Open the PDF file in read-binary mode
with open(pick_file(), 'rb') as file:
    # Create a PyPDF2 PdfFileReader object
    reader = PyPDF2.PdfReader(file)

    # Read the text from each page and append it to the raw_text variable
    raw_text = ''
    for i in range(len(reader.pages)):
        page = reader.pages[i]
        text = page.extract_text()
        if text:
            raw_text += text


In [ ]:
# Open the txt file in read mode
with open(pick_file(), 'r') as file:
    # Read the text from the file and store it in the raw_text variable
    raw_text = file.read()

In [ ]:
import csv, json
with open(pick_file(), 'r', encoding='utf-8') as file:
    # create a CSV reader object
    reader = csv.DictReader(file)
    
    # iterate over the rows in the CSV file and store them in a list
    rows = []
    for row in reader:
        rows.append(row)

    # convert list of dictionaries to JSON-formatted string
    raw_text = json.dumps(rows)

In [ ]:
raw_text[:200]

In [ ]:
raw_text.replace(";","")

In [ ]:
with open("output.txt", "w") as file:
    json.dump(raw_text.replace(";",""), file, ensure_ascii=False)

: 

In [ ]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [ ]:
len(texts)

In [ ]:
texts[0]

In [ ]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [ ]:
docsearch = FAISS.from_texts(texts, embeddings)

In [ ]:
docsearch

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [ ]:
query = input("Question: ")
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)